# Running GWAS using limix
## Using our phenotype from the first notebook, we are going to run GWAS.

### Steps of this notebook:
Preparing input data  
Running GWAS using the limix library for python  
Outputting the results.

Limix is freely available [here](https://github.com/limix/limix) and has an extensive [documentation](https://limix.readthedocs.io/).

# 1.  Initial setup steps

## 1a. Set up the environment

In [1]:
import pandas as pd
import numpy as np
import os
import h5py
from limix.qtl import scan
from bisect import bisect

## 1b. Define variables

In [2]:
# phenotype file
#pheno_file = './data/subset_flowering_time_16.csv'
pheno_file = './data/subset_flowering_time_16.csv'

# genotype file
geno_file = './data/subset_all_chromosomes_binary.hdf5'

# kinship matrix file
kin_file = './data/kinship_ibs_binary_mac5.h5py'

# minor allele frequency threshold - we will talk more about why we do this in the next notebook!
# 10% is the standard cutoff we apply to *Arabidopsis thaliana* GWAS, but this may be different in your organism of choice.
MAF_thrs = 0.1

# output file for results with K correction
output_file = './results/subset_flowering_time_16_gwas.csv'

# output file for results without K correction
output_file_nc = './results/subset_flowering_time_16_gwas_nc.csv'

# 2. Load input for GWAS

As a reminder, the linear mixed model for GWAS is:

$
\begin{align}
Y   = X\beta + K + \epsilon
\end{align}
$


We need three variables to run GWAS in limix:
1.  Y = Phenotype matrix (Y in model)
2.  G = Genotype matrix (X in model)
3.  K = K matrix (K in model)

limix will use this input to estimate the association p-value and effect size
($
\begin{align}
\beta
\end{align}
$) for each SNP.



The first step to generating these three variables is to load the data.
### Please Note!
This script is written to handle data from the *Arabidopsis thaliana* 1001 genome project.  If you are running GWAS using another organism, you will more than likely start with data that is formatted differently!  If this is the case, some parts of this code *will not* work for you out of the box.  However, if you understand the __steps__ that this code takes, you will be able to adapt it to suit your specific input data.

Therefore, I would like you to __focus on the general approach__ to generating the Y, G, and K matrices rather than trying to understand every line of code.

## 2a. Load phenotypes
The phenotype data are stored in a 2-column .csv file.
The first column specifies the accession identifier ("ecotypeid"), the second column contains the phenotype value.  Our flowering time dataset has 200 accessions.

In [3]:
# load phenotype data
pheno = pd.read_csv(pheno_file, index_col = 0)

# encode the index (the accessions column) to UTF8 so it matches encoding of accessions in the genotype data.
# this will allow us to compare the two datasets in section 3a. 
pheno.index = pheno.index.map(lambda x: str(int(x)).encode('UTF8'))

In [4]:
# remove accessions with missing or non numerical data
pheno = pheno.dropna()

In [5]:
# does pheno match our expectations?
print(pheno.shape)
print(pheno.head(n=5))

(200, 1)
           flowering_time_16
ecotypeid                   
b'770'                 72.25
b'801'                 88.25
b'991'                106.75
b'1062'                68.25
b'1367'                88.75


The phenotypes are loaded.

## 2b. Load genotypes
The genotypes we're going to use are a subset of SNPs obtained from whole-genome resequencing of 1,135 *Arabidopsis thaliana* accessions ([1001 genomes](http://1001genomes.org/)).

Genotype data is stored as an [hdf5](https://www.h5py.org/) file, which is a composite data type.  This means that one hdf5 file stores multiple related data sets. 

The genotype hdf5 file we are using here consists of three data sets:
1.  'accessions' contains the accession identifiers.
2.  'positions' provides the SNP positions.
3.  'snps' gives the SNP calls themselves. SNPs are coded as 0 for reference allele and 1 for alternate allele.

The 'positions' dataset is also associated with a small file called an attribute which provides information about the chromosome location ('chr_index').  We will use the attribute later when outputting GWAS results.

(If you are interested in learning more about how to use hdf5 files, check out https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/ after class for a more detailed introduction.)


In [6]:
# load genotype data
geno_hdf = h5py.File(geno_file, 'r')
# structure of hdf5 file
# does geno_hdf match our expectations? Here, "key" refers to the three different data sets
for key in geno_hdf.keys():
    print(key)
    print(geno_hdf[key].shape)
    print(geno_hdf[key][0:10])

accessions
(1135,)
[b'88' b'108' b'139' b'159' b'265' b'350' b'351' b'403' b'410' b'424']
positions
(1070995,)
[ 55 101 139 203 237 291 332 375 431 502]
snps
(1070995, 1135)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]


## 2c.  Load K matrix
The kinship matrix we are using is an IBS (identity by state) matrix calculated with the complete SNP dataset (rather than the SNP subset we are using).  As discussed in lecture, there any many options for calculating a K matrix, but we are using a pre-calculated matrix to save ourselves the long computation time.

The input file is also an hdf5 file.  It contains 2 datasets:
1. 'accessions' gives the accession identifiers
2. 'kinship' is the kinship matrix.  (In this case, it is calculated for all 1135 accessions in the complete 1001 genomes dataset.)

In [7]:
kin_hdf = h5py.File(kin_file, 'r')

print(kin_hdf['accessions'].shape)
print(kin_hdf['accessions'][0:5])
print(kin_hdf['kinship'].shape)
print(kin_hdf['kinship'][0:5])

(1135,)
[b'88' b'108' b'139' b'159' b'265']
(1135, 1135)
[[7.32260312 6.44804411 6.44906758 ... 6.33012188 6.33045141 6.33019166]
 [6.44804411 7.32260312 7.30934451 ... 6.32966248 6.3303099  6.33010831]
 [6.44906758 7.30934451 7.32260312 ... 6.33023237 6.33063943 6.33054639]
 [6.44129268 6.38031273 6.37999871 ... 6.34332622 6.34272144 6.34269043]
 [6.33677639 6.34437683 6.34471411 ... 6.2482848  6.24887795 6.24868411]]


# 3.  Generating matrices for GWAS
Now that the data have been loaded, we need to do some final manipulations to generate the appropriate input matrices.

We have __*three main objectives*__ here:

1.  Since limix will not work with missing data, we need to make sure that all three matrices include the __same set of accessions__.
2.  We also need to make sure that the accessions appear in the __same order__ in all three matrices.
3.  We will also remove any SNPs that don't meet our __minor allele frequency__ threshold. (We will return to the reason for this step in the next notebook.)

Again, this code will work for the 1001 genomes data out of the box, but will likely need to be modified for other data.  Therefore, focus on the __steps__ rather than on each individual line of code.

## 3a.  Generate phenotype matrix (Y)

We now have the phenotypes stored as the variable called 'pheno'.  To make the phenotype matrix (Y), we need to __*remove non-genotyped accessions*__ and __*order the data*__ to match the genotype data.

In [8]:
# remove non-genotyped accessions from phenotype
acn_genotyped = [acn for acn in pheno.index if acn in geno_hdf['accessions'][:]]
# subset phenotype data
pheno = pheno.loc[acn_genotyped]
# order genotypes in phenotype according SNP-matrix
acn_indices = [np.where(geno_hdf['accessions'][:] == acn)[0][0] for acn in pheno.index]
acn_indices.sort()
acn_order = geno_hdf['accessions'][acn_indices]
pheno = pheno.loc[acn_order]
# transform to a numpy phenotype matrix (Y)
Y = pheno.to_numpy()

The phenotype matrix (Y) is now ready.

## 3b. Generate genotype matrix (G)
Now we can finish the genotype matrix (G).  Here we remove:
1. accessions that are not phenotyped 
2. SNPs with a minor allele frequency below our threshold.

In [9]:
# subset SNP matrix for phenotyped genotypes
# be patient - this step can take a few minutes
G = geno_hdf['snps'][:, acn_indices]

In [10]:
# remove SNPs with minor allele frequency below set threshold
# count allele 1 and 0 for each SNP
AC1 = G.sum(axis = 1)
AC0 = G.shape[1] - AC1
AC = np.vstack((AC0,AC1))
# define the minor allele for each position
MAC = np.min(AC, axis = 0)
# calculate minor allele frequency
MAF = MAC/G.shape[1]
# select SNPs with MAF above threshold 
SNP_indices = np.where(MAF >= MAF_thrs)[0]
SNPs_MAF = MAF[SNP_indices]
G = G[SNP_indices, :]

# transpose SNP-matrix into accessions x SNPs matrix
G = G.transpose()
geno_hdf.close()


The genotype matrix (G) is now ready.

## 3c. Generate kinship matrix (K)
To finish the kinship array, subset for accessions that are phenotyped and genotyped.

In [11]:
# subset kinship matrix for phenotyped and genotyped accessions
acn_indices = [np.where(kin_hdf['accessions'][:] == acn)[0][0] for acn in pheno.index]
acn_indices.sort()
K = kin_hdf['kinship'][acn_indices, :][:, acn_indices]
kin_hdf.close()

The kinship matrix (K) is now ready.

# 4. Check input variables

Now we have all three of our input variables.  If you are running GWAS using something other than the 1001 genomes data, __*these are the matrices that you need to generate to run limix*__.  Let's quickly look at these variables to make sure we understand their format.  The number of accessions should be the same in all three matrices.

### 4a. Y matrix (phenotypes)
*Y is a numpy array.
The number of rows = number of accessions.
The number of columns = 1.*

In [12]:
print(type(Y))
print(Y.shape)
print(Y[0:5])

<class 'numpy.ndarray'>
(170, 1)
[[ 88.25      ]
 [106.75      ]
 [ 68.25      ]
 [ 69.33333333]
 [ 69.75      ]]


### 4b. G matrix (genotypes)
*G is a numpy array.  The number of rows = number of accessions.  The number of columns = the number of snps.*

In [13]:
print(type(G))
print(G.shape)
print(G[0:5])

<class 'numpy.ndarray'>
(170, 125066)
[[0 0 0 ... 0 1 1]
 [1 1 1 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


### 4c. K matrix (K matrix)
*K is a numpy array.  The number of rows and columns = the number of accessions*

In [14]:
print(type(K))
print(K.shape)
print(K)

<class 'numpy.ndarray'>
(170, 170)
[[7.32260312 6.34682696 6.33634413 ... 6.34759263 6.32583997 6.35192106]
 [6.34682696 7.32260312 6.42985228 ... 6.3919586  6.2824742  6.35606923]
 [6.33634413 6.42985228 7.32260312 ... 6.36691067 6.29265853 6.33635963]
 ...
 [6.34759263 6.3919586  6.36691067 ... 7.32260312 6.31910792 6.37552489]
 [6.32583997 6.2824742  6.29265853 ... 6.31910792 7.32260312 6.32155224]
 [6.35192106 6.35606923 6.33635963 ... 6.37552489 6.32155224 7.32260312]]


# 5.  Run GWAS
We are going to run two GWAS with our phenotype of interest.  The first one will include a population structure correction (i.e. the model will include the K matrix) and the second one will not (we will compare the results in the next notebook).

## 5a.  Run limix with K correction

Now that we have phenotype (Y), genotype (G), and K (K) matrices, we can run GWAS using the 'scan' function from limix.

In [15]:
r = scan(G, Y, K = K, lik = 'normal', M = None, verbose = True)

============================= QTL analysis session starts ==============================

Normalising input... 
done (1.13 second).


LMM: 55it [00:00, 3846.64it/s]
Results: 100%|██████████| 125066/125066 [00:03<00:00, 36846.87it/s]


Hypothesis 0

𝐲 ~ 𝓝(𝙼𝜶, 422.851⋅𝙺 + 0.000⋅𝙸)

M     = ['offset']
𝜶     = [86.3851492]
se(𝜶) = [51.39663417]
lml   = -740.1760193587634

Hypothesis 2

𝐲 ~ 𝓝(𝙼𝜶 + G𝛃, s(422.851⋅𝙺 + 0.000⋅𝙸))

          lml       cov. effsizes   cand. effsizes
--------------------------------------------------
mean   -7.397e+02       8.611e+01        5.201e-01
std     7.455e-01       1.722e+00        4.118e+00
min    -7.402e+02       6.975e+01       -2.084e+01
25%    -7.401e+02       8.547e+01       -2.129e+00
50%    -7.399e+02       8.630e+01        4.310e-01
75%    -7.395e+02       8.687e+01        3.080e+00
max    -7.275e+02       1.001e+02        3.051e+01

Likelihood-ratio test p-values

       𝓗₀ vs 𝓗₂ 
----------------
mean   4.976e-01
std    2.907e-01
min    4.893e-07
25%    2.454e-01
50%    4.965e-01
75%    7.491e-01
max    1.000e+00


/Users/danielefiliault/opt/anaconda3/envs/limix/lib/python3.7/site-packages/limix/qtl/_result/_st_result.py:115: FutureWarning: `item` has been deprecated and will be removed in a future version
  v0 = self.h0.variances["fore_covariance"].item()
/Users/danielefiliault/opt/anaconda3/envs/limix/lib/python3.7/site-packages/limix/qtl/_result/_st_result.py:116: FutureWarning: `item` has been deprecated and will be removed in a future version
  v1 = self.h0.variances["back_covariance"].item()


====================== QTL analysis session ends in 6.62 seconds =======================

This is the output generated by limix.  Limix has run the linear mixed model for each SNP, and this output summarizes the range of values calculated across all these tests.
Although this output is quite detailed, we are primarily interested in two values from limix:

1. __P-value__ - This is the p-value for the test that a SNP is associated with the phenotype.  Limix calculates this as likelihood-ratio test between our full GWAS model (H2 in the output) and a model that doesn't contain the genotype term (H0 in the output).  The range of these values across all tested SNPs is given under the heading "Likelihood-ratio test p-values".
2. __Effect size__ - This is the effect of a SNP in the linear model (the beta in the GWAS equation).   The range of these values across all tested SNPs is given as "cand. effsizes" in the table below "Hypothesis 2".

In the next section, we will output the p-values and effect sizes for all SNPs tested.

## 5b.  Output results for limix with K correction

We will look at the results of these GWAS is the next notebook.  This cell saves the GWAS results as a .csv file that gives chromosome (chr), position (pos), p-value (pvalue), minor allele frequency (maf), and effect size (GVE) for each SNP.

In [16]:
# save results

# get chromosomes, positions, and minor allele frequencies
# you would need to recode this section if working with something other than the 1001 genomes data
geno_hdf = h5py.File(geno_file, 'r')
chr_index = geno_hdf['positions'].attrs['chr_regions']
chromosomes = [bisect(chr_index[:, 1], snp_index) + 1 for snp_index in SNP_indices]
positions_all = geno_hdf['positions'][:]
positions = [positions_all[snp] for snp in SNP_indices]
mafs = SNPs_MAF #from section 3b

# get p-value and effect size
# these variables are output from limix and should work regardless of initial input data format
#extract p-values
pvalues = r.stats.pv20.tolist()
#extract effect sizes
effsizes = r.effsizes['h2']['effsize'][r.effsizes['h2']['effect_type'] == 'candidate'].tolist()

gwas_results = pd.DataFrame(list(zip(chromosomes, positions, pvalues, mafs, effsizes)), columns = ['chr', 'pos', 'pvalue', 'maf', 'GVE'])
gwas_results.to_csv(output_file, index = False)
geno_hdf.close()

## 5c.  Run limix without K correction

We will also run limix without population structure correction (i.e. without K in the model) so we can compare results in the next notebook.

In [17]:
r_nc = scan(G, Y, lik="normal", M=None, verbose=True)

============================= QTL analysis session starts ==============================

Normalising input... 
done (0.53 seconds).


Results: 100%|██████████| 125066/125066 [00:03<00:00, 38026.10it/s]


Hypothesis 0

𝐲 ~ 𝓝(𝙼𝜶, 653.950⋅𝙸)

M     = ['offset']
𝜶     = [76.3995098]
se(𝜶) = [1.96131695]
lml   = -792.277165089571

Hypothesis 2

𝐲 ~ 𝓝(𝙼𝜶 + G𝛃, s(653.950⋅𝙸))

          lml       cov. effsizes   cand. effsizes
--------------------------------------------------
mean   -7.902e+02       7.578e+01        1.860e+00
std     2.961e+00       3.724e+00        9.727e+00
min    -7.923e+02       4.870e+01       -3.737e+01
25%    -7.921e+02       7.403e+01       -4.875e+00
50%    -7.914e+02       7.608e+01        1.325e+00
75%    -7.896e+02       7.756e+01        8.047e+00
max    -7.629e+02       1.100e+02        4.610e+01

Likelihood-ratio test p-values

       𝓗₀ vs 𝓗₂ 
----------------
mean   2.955e-01
std    3.058e-01
min    1.721e-14
25%    2.071e-02
50%    1.785e-01
75%    5.252e-01
max    1.000e+00


====================== QTL analysis session ends in 5.56 seconds =======================

This output is in the same format as that of section 5a.

## 5d.  Output results for limix without K correction
We will look at the results of these GWAS is the next notebook.  This cell saves the GWAS results as a .csv file that gives chromosome (chr), position (pos), p-value (pvalue), minor allele frequency (maf), and effect size (GVE) for each SNP.

In [18]:
# save results
# link chromosome and position to p-values and effect sizes
geno_hdf = h5py.File(geno_file, 'r')
chr_index = geno_hdf['positions'].attrs['chr_regions']
chromosomes = [bisect(chr_index[:, 1], snp_index) + 1 for snp_index in SNP_indices]
positions_all = geno_hdf['positions'][:]
positions = [positions_all[snp] for snp in SNP_indices]
mafs = SNPs_MAF #from section 3b

pvalues = r_nc.stats.pv20.tolist()
effsizes = r_nc.effsizes['h2']['effsize'][r_nc.effsizes['h2']['effect_type'] == 'candidate'].tolist()

gwas_results = pd.DataFrame(list(zip(chromosomes, positions, pvalues, mafs, effsizes)), columns = ['chr', 'pos', 'pvalue', 'maf', 'GVE'])
gwas_results.to_csv(output_file_nc, index = False)
geno_hdf.close()

 ### In this notebook, we have learned about input data, run GWAS, and output GWAS results.
 ### Let's move on to 3_GWAS_interpretation.ipynb, where we will explore these results.